# ExpertFusion: A Mixture-of-Experts System for Market Prediction

This notebook demonstrates how to use the ExpertFusion model for market prediction using a Mixture-of-Experts (MoE) architecture.

In [ ]:
# Clone the repository
!git clone https://github.com/DavidZSGao/ExpertFusion.git
%cd ExpertFusion

# Install dependencies
!pip install -r requirements.txt

In [ ]:
import os
from data_loader import load_cached_data
from moe_model import MoEModel
from experts import MacroExpert, FundamentalExpert, NewsExpert, TechnicalExpert, RiskExpert

# Set your API keys here
os.environ['OPENAI_API_KEY'] = 'your-api-key'
os.environ['WRDS_USERNAME'] = 'your-username'

In [ ]:
# Initialize experts
experts = [
    MacroExpert(),
    FundamentalExpert(),
    NewsExpert(),
    TechnicalExpert(),
    RiskExpert()
]

# Create MoE model
model = MoEModel(experts)

# Make predictions
predictions = model.predict('AAPL')
print(f"Predictions: {predictions}")